
# DCDE 11/14/19 dry run

## Topics
  * prerequisites
  * recipes, templates
  * lessons learned
  * components we're using
  * Participants
  * How accounts are created
  * Live demo


## Demo steps  

  - Prepare for demo
    - Log out all the things
    - Activate Globus endpoints
  - Demo begins
    - Log in to jupyterhub via cilogon
    - Talk about single DCDE identity used across sites, how they're set up for it
    - Talk about 3 sites participating - DCDE set up w/ oauth_ssh, Globus, mix of Condor & Slurm
    - Load demo page
    - Introduce Relion - say we're using containers, singlarity at each site
    - Show data set at bnl - say we've got it staged to other prerequisites
    - Talk about parsl -- we're leveraging it to run across a distributed,  mixed environment
    - Run import(?) at BNL.  
    - Sync data out to ORNL for (motioncorr or ctffind) -- something quick
    - Sync data back to BNL via Globus
    - Sync data out to ANL
    - Run autopick 3d refine(?) at  anl
    - pull data back to bnl
    - Show pictures w/ nglview at BNL

*note: clean relion dataset is at gssh.lcrc.anl.gov:/home/dcowley/dcde-sc19-relion-data-clean.tgz.  There may be copies elsewhere...*

## list of steps/outputs:

| Target Site | job step | output type |  Output treatment | Approx. time|
| ----- | -----  | ----- | ----- | ---|
| ORNL |autopick | | | | |
| ORNL |extract | | | |
| ? | ctffind |  | | |
| ? | autopick |  | | |
| ANL | 3d refine? - view w/ NGL |   | | | 

##  Investigate (see sc19-screenply-cruft.md):

  * Did I do motioncor on Cascade w/ GPGPUs? I think I did.  
    * Is unblur in the singularity container?
  * Will relion_display work in any way?
  * Can we have some canned pictures?  Capture shots from X display or Chimera or something


In [ ]:
import sys
print(sys.path)

In [1]:
""" Here is all the parsl/Globus setup """


import parsl
import os
from parsl.config import Config


from parsl.channels import OAuthSSHChannel
from parsl.providers import CondorProvider
from parsl.providers import SlurmProvider
from parsl.launchers import SrunLauncher
from parsl.executors import HighThroughputExecutor
from parsl.addresses import address_by_hostname
from parsl.app.app import bash_app
from parsl.app.app import python_app

#parsl.set_stream_logger()

anl_config = Config(
    app_cache=True,
    checkpoint_files=None,
    checkpoint_mode=None,
    checkpoint_period=None,
    data_management_max_threads=10,
    executors=[HighThroughputExecutor(
        address='130.199.185.13',
        cores_per_worker=1.0,
        heartbeat_period=30,
        heartbeat_threshold=120,
        interchange_port_range=(50000, 51000),
        label='anl-slurm',
        launch_cmd='process_worker_pool.py {debug} {max_workers} -p {prefetch_capacity} -c {cores_per_worker} -m {mem_per_worker} --poll {poll_period} --task_url={task_url} --result_url={result_url} --logdir={logdir} --block_id={{block_id}} --hb_period={heartbeat_period} --hb_threshold={heartbeat_threshold} ',
        managed=True,
        max_workers=1,
        #mem_per_worker=None,
        poll_period=10,
        prefetch_capacity=0,
        interchange_address='10.70.128.9', #this is the address worker talk to inetrchange(head node)
        provider=SlurmProvider(
            'debug',
            channel=OAuthSSHChannel(
                'gssh.lcrc.anl.gov',
                envs={},
                port=2222,
                script_dir='/home/dcowley/ornl-parsl-scripts',
                username='dcowley'
            ),
            cmd_timeout=10,
            exclusive=True,
            init_blocks=1,
            # launcher=SingleNodeLauncher(),
            max_blocks=1,
            min_blocks=1,
            move_files=True,
            nodes_per_block=1,
            parallelism=0.0,
            scheduler_options='#SBATCH -A dcde\n#SBATCH -t 0:20:00\n#SBATCH -N 1\n#SBATCH --ntasks-per-node=36\n#SBATCH -J relion-autopick\n#SBATCH -p bdwall\n#SBATCH -D /blues/gpfs/home/dcowley/relion-bootstrap\n#SBATCH -o relion-autopick.%j.out\n#SBATCH -e relion-autopick.%j.err',
            walltime='00:10:00',
            #worker_init='source /home/dcde1000001/dcdesetup.sh'
            worker_init='source /lcrc/project/DCDE/setup.sh;  source activate /lcrc/project/DCDE/envs/dcdeRX; export I_MPI_FABRICS=shm:tmi'
        ),
        storage_access=[],
        suppress_failure=False,
        worker_debug=True,
        worker_logdir_root='/home/dcowley/parsl_scripts/logs',
        worker_port_range=(50000, 51000),
        #worker_ports=None,
        working_dir='/home/dcowley/parsl_scripts'
    )],
    lazy_errors=True,
    monitoring=None,
    retries=0,
    run_dir='runinfo',
    strategy='simple',
    usage_tracking=False
)

bnl_config = Config(
    app_cache=True,
    checkpoint_files=None,
    checkpoint_mode=None,
    checkpoint_period=None,
    data_management_max_threads=10,
    executors=[HighThroughputExecutor(
        #address='127.0.0.1',
        address='130.199.185.13',
        cores_per_worker=1,
        heartbeat_period=30,
        heartbeat_threshold=120,
        interchange_port_range=(50000, 51000),
        label='bnl-condor',
        launch_cmd='process_worker_pool.py {debug} {max_workers} -p {prefetch_capacity} -c {cores_per_worker} -m {mem_per_worker} --poll {poll_period} --task_url={task_url} --result_url={result_url} --logdir={logdir} --block_id={{block_id}} --hb_period={heartbeat_period} --hb_threshold={heartbeat_threshold} ',
        mem_per_worker=4,
        managed=True,
        max_workers=1,
        poll_period=10,
        prefetch_capacity=0,
        interchange_address='130.199.185.9', #this is the address worker talk to inetrchange(head node)
        provider=CondorProvider(
            channel=OAuthSSHChannel(
                'spce01.sdcc.bnl.gov',
                envs={},
                port=2222,
                script_dir='/sdcc/u/dcde1000006/parsl_scripts',
                username='dcde1000006'
            ),
            environment={},
            init_blocks=1,
            # launcher=SingleNodeLauncher(),
            max_blocks=1,
            min_blocks=1,
            nodes_per_block=1,
            #parallelism=1,
            parallelism=0,
            project='',
            #Trying this Requirements directive per Dong's instructions:
            #requirements='regexp("^sp[oa]", machine)',
            scheduler_options='accounting_group = group_sdcc.main \nRequirements = (regexp("^sp[oa]", machine))',
            transfer_input_files=[],
            walltime='00:30:00',
            #worker_init='source /sdcc/u/dcde1000001/dcdesetup.sh'
            worker_init='source /hpcgpfs01/work/dcde/setup.sh; source activate dcdeRX'
        ),
        storage_access=[],
        suppress_failure=False,
        worker_debug=True,
        worker_logdir_root='/sdcc/u/dcde1000006/parsl_scripts/logs',
        worker_port_range=(50000, 51000),
        #worker_port_range=(5000, 5100),   # per John H's message 8/29/19
        worker_ports=None,
        working_dir='/sdcc/u/dcde1000006/parsl_scripts'
    )],
    lazy_errors=True,
    monitoring=None,
    retries=0,
    run_dir='runinfo',
    strategy='simple',
    usage_tracking=False
)

ornl_config = Config(
    app_cache=True,
    checkpoint_files=None,
    checkpoint_mode=None,
    checkpoint_period=None,
    data_management_max_threads=10,
    executors=[HighThroughputExecutor(
        address='130.199.185.13',
        cores_per_worker=1.0,
        heartbeat_period=30,
        heartbeat_threshold=120,
        interchange_port_range=(50000, 51000),
        label='ornl-slurm',
        launch_cmd='process_worker_pool.py {debug} {max_workers} -p {prefetch_capacity} -c {cores_per_worker} -m {mem_per_worker} --poll {poll_period} --task_url={task_url} --result_url={result_url} --logdir={logdir} --block_id={{block_id}} --hb_period={heartbeat_period} --hb_threshold={heartbeat_threshold} ',
        managed=True,
        max_workers=1,
        #mem_per_worker=None,
        poll_period=10,
        prefetch_capacity=0,
        interchange_address='128.219.185.39', #this is the address worker talk to inetrchange(head node)
        provider=SlurmProvider(
            'debug',
            channel=OAuthSSHChannel(
                'dcde-ext.ornl.gov',
                envs={},
                port=2222,
                script_dir='/home/dcde1000006/ornl-parsl-scripts',
                username='dcde1000006'
            ),
            cmd_timeout=10,
            exclusive=True,
            init_blocks=1,
            # launcher=SingleNodeLauncher(),
            max_blocks=1,
            min_blocks=1,
            move_files=True,
            nodes_per_block=1,
            parallelism=0.0,
            scheduler_options='#SBATCH -D /nfs/scratch/relion-bootstrap\n#SBATCH -o relion-autopick.%j.out\n#SBATCH -e relion-autopick.%j.err',
            walltime='00:10:00',
            worker_init='source /nfs/scratch/dcde1000012/RX.sh'
        ),
        storage_access=[],
        suppress_failure=False,
        worker_debug=True,
        worker_logdir_root='/nfs/scratch/dcde1000006/parsl_scripts/logs',
        worker_port_range=(50000, 51000),
        #worker_ports=None,
        working_dir='/nfs/scratch/dcde1000006/parsl_scripts'
    )],
    lazy_errors=True,
    monitoring=None,
    retries=0,
    run_dir='runinfo',
    strategy='simple',
    usage_tracking=False
)

ANL_EP = '57b72e31-9f22-11e8-96e1-0a6d4e044368'
BNL_EP = '23f78cc8-41e0-11e9-a618-0a54e005f950'
EMSL_EP = 'e133a52e-6d04-11e5-ba46-22000b92c6ec'
ORNL_EP = '57230a10-7ba2-11e7-8c3b-22000b9923ef'


In [ ]:
""" Set up Globus auth"""
import subprocess
import json
from globus_sdk import (NativeAppAuthClient, TransferClient,
                        RefreshTokenAuthorizer, TransferData)
from globus_sdk.exc import GlobusAPIError

authout = subprocess.run(['/usr/local/anaconda3/bin/parsl-globus-auth'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print (authout.stdout)
print (authout.stderr)
# Perform a Globus directory transfer, reusing refresh tokens we've already obtained for PARSL.
# Note this is NOT a PARSL transfer


def load_tokens_from_file(filepath):
    """Load a set of saved tokens."""
    with open(filepath, 'r') as f:
        tokens = json.load(f)

    return tokens

def save_tokens_to_file(filepath, tokens):
    """Save a set of tokens for later use."""
    with open(filepath, 'w') as f:
        json.dump(tokens, f)


def update_tokens_file_on_refresh(token_response):
    """
    Callback function passed into the RefreshTokenAuthorizer.
    Will be invoked any time a new access token is fetched.
    """
    save_tokens_to_file(globus_tokens, token_response.by_resource_server)

dcde_parsl_client_id = '8b8060fd-610e-4a74-885e-1051c71ad473'

globus_tokens='/home/dcde1000006/.parsl/.globus.json'

# First authorize using those refresh tokens:

try:
    tokens = load_tokens_from_file(globus_tokens)

except:
    print("Valid refresh tokens not found in {}.  Unable to authorize to Globus.  Exiting!".format(globus_tokens))
    sys.exit(-1)


transfer_tokens = tokens['transfer.api.globus.org']

try:
    auth_client = NativeAppAuthClient(client_id=dcde_parsl_client_id)
except:
    print ("ERROR: Globus NativeAppAuthClient() call failed!  Unable to obtain a Globus authorizer!")
    sys.exit(-1)

authorizer = RefreshTokenAuthorizer(
    transfer_tokens['refresh_token'],
    auth_client,
    access_token=transfer_tokens['access_token'],
    expires_at=transfer_tokens['expires_at_seconds'],
    on_refresh=update_tokens_file_on_refresh)

try:
    tc = TransferClient(authorizer=authorizer)
except:
    print ("ERROR: TransferClient() call failed!  Unable to call the Globus transfer interface with the provided auth info!")
    sys.exit(-1)





Now we should have a transfer client with auth, We can set up one or many transfers.  Remember each TransferData object has a specific src/dest, and we need to build in a list of files/dirs with add_item().

Now Check to see we have parsl configs loaded for remote execution:

## The DCDE

(Talk about DCDE environment, single identity, use of multiple sites with that one identity.  Compute/data at 3 sites: ANL, BNL, ORNL.  Talk about use case: Observational science with computing requirements, specifically Cryo-electron microscopy.  We have a body of data, need to do various computing steps on it to go from raw data to 3d reconstructions of proteings)

## The application & demonstration

(Talk about Relion, which processes the Cryo-EM data. We will run the application in a singularity container at multiple compute sites, using Globus to sync the data across sites.  The workflow will be controlled by the parsl library of python functions, which we will use to run distributed HPC-style jobs)

**Demo step: Show data set at BNL**

In [ ]:
!ls -lht /hpcgpfs01/scratch/dcde1000006/sc19-data

**Demo step: Run import at BNL**  *Note: The short bnl job that i have right now is relion_postprocess*

In [2]:
parsl.clear()

#parsl.set_stream_logger()
parsl.load(bnl_config)
bnl_dfk = parsl.dfk()

@bash_app
def relion_import(job_dir=None, stdout=None, stderr=None, mock=False):
    """
    Parameters
    ----------
    mock : (Bool)
       when mock=True
    """
    cmd_line = '''#!/bin/bash -l

export DATAROOT=/hpcgpfs01/scratch/dcde1000006/sc19-data
export RELION_SIMG=/sdcc/u/dcde1000006/relion_singv26.simg

export MOVIESTAR=${{DATAROOT}}/Import/job001/movies.star
export INSTAR=${{DATAROOT}}/CtfFind/job003/micrographs_ctf.star
export REFSTAR=${{DATAROOT}}/Select/job007/class_averages.star
export PICKDIR=${{DATAROOT}}/AutoPick/job010/

cd ${{DATAROOT}}
echo -n "working directory: "
pwd
#set -v

singularity exec  -B /hpcgpfs01:/hpcgpfs01 ${{RELION_SIMG}} relion_star_loopheader rlnMicrographMovieName > ${{MOVIESTAR}}
singularity exec  -B /hpcgpfs01:/hpcgpfs01 ${{RELION_SIMG}} ls Micrographs/*.mrcs >> ${{MOVIESTAR}}
    '''
    if mock:
        return '''tmp_file=$(mktemp);
cat<<EOF > $tmp_file
{}
EOF
cat $tmp_file
        '''.format(cmd_line)
    else:
        return cmd_line


relion_stdout=os.path.join(bnl_config.executors[0].working_dir, 'relion-bnl-import.out')
relion_stderr=os.path.join( bnl_config.executors[0].working_dir, 'relion-bnl-import.err')

local_logdir='/hpcgpfs01/scratch/dcde1000006/sc19-data/parsl-outputs'
local_logfile=os.path.join(local_logdir, 'relion-bnl-import.out')

try:
    os.remove(relion_stdout)
except OSError:
    pass
except FileNotFoundError:
    pass
try:
    os.remove(relion_stderr)
except OSError:
    pass
except FileNotFoundError:
    pass
try:
    os.remove(local_logfile)
except OSError:
    pass
except FileNotFoundError:
    pass


print ('job setup: stdout = {}\nstderr = {}'.format(relion_stdout,relion_stderr))
# parsl.set_stream_logger()

x = relion_import(job_dir=bnl_config.executors[0].working_dir, stdout=relion_stdout, stderr=relion_stderr, mock = False )
print('relion_import() invoked, now waiting...')
x.result()

if x.done():
    bnl_dfk.executors['bnl-condor'].provider.channel.pull_file(relion_stdout, local_logdir)
    with open(local_logfile, 'r') as f:
        print(f.read())


job setup: stdout = /sdcc/u/dcde1000006/parsl_scripts/relion-bnl-import.out
stderr = /sdcc/u/dcde1000006/parsl_scripts/relion-bnl-import.err
relion_import() invoked, now waiting...
working directory: /hpcgpfs01/scratch/dcde1000006/sc19-data



In [ ]:
!ls -lt /hpcgpfs01/scratch/dcde1000006/sc19-data/Import/job001

In [ ]:
exit()

**Demo step: Sync data from BNL to ORNL via Globus**

**Demo step: Run extract & autopick steps at ORNL**

**Demo step: Sync new data back from ORNL to BNL**

**Demo step: Sync new data back from ORNL to BNL, sync BNL to ANL**

**Demo step: Sync new data back from ORNL to BNL, sync BNL to ANL**

**Demo step: (Skipping ahead) run 3d refinement (parallel?) at ANL**

**Demo step: Sync new data back from ANL to BNL**

**Demo step: Show 3d reconstruction in nglview**

# End of Demo

## Move me: Sync fresh data set from ANL to BNL

In [ ]:

tdata = TransferData(tc, ANL_EP, BNL_EP,
                     label="DCDE Relion transfer",
                     sync_level="size")

tdata.add_item('/blues/gpfs/home/dcowley/dcde-sc19-relion-data-clean.tgz',
            '/hpcgpfs01/scratch/dcde1000006/dcde-sc19-relion-data-clean.tgz')

transfer_result = tc.submit_transfer(tdata)

print("task_id =", transfer_result["task_id"])


while not tc.task_wait(transfer_result['task_id'], timeout=1200, polling_interval=10):
    print(".", end="")
print("\n{} completed!".format(transfer_result['task_id']))

os.listdir(path='/hpcgpfs01/scratch/dcde1000006/')

In [ ]:
!ls -l /hpcgpfs01/scratch/dcde1000006/